In [52]:
import pandas as pd

# Load JSON data
df = pd.read_json('KitTickets.json')
# drop unwanted features
df.drop(['Id', 'RefTicketId', 'TicketId', 'TicketMapId', 'id', 'Attachments', 'ChangesHistory', 'Comments', '_rid', '_self', '_etag', '_attachments', 'ModifiedDateUtc', 'IsActive', 'IsDeleted', '_ts'], axis=1, inplace=True)


In [53]:
# Pull-out the features from TicketJson
Ticket_object = df['TicketJson']

df2 = pd.DataFrame(Ticket_object.tolist())  # Convert the Series of dictionaries to a DataFrame
print(df2.columns) 

Index(['sys_id', 'number', 'u_business_caller', 'short_description',
       'description', 'priority', 'state', 'incident_state', 'u_description',
       'start_date', 'end_date', 'justification', 'implementation_plan',
       'backout_plan', 'risk', 'type', 'u_manager_approval', 'u_cab_approve',
       'key', 'id', 'status', 'reported'],
      dtype='object')


In [54]:
# Include the pulled features to the dataframe
df['short_description'] = df2['short_description']
df['priority'] = df2['priority']


In [55]:
df.drop(['TicketJson'], axis='columns', inplace=True)

In [56]:
# Extract the date from the 'CreatedDateUtc' feature
df['CreatedDateUtc_Short'] = df['CreatedDateUtc'].str[:10]
print(df['CreatedDateUtc_Short'].unique())
print(df['priority'].unique())

['2024-04-16' '2024-04-18' '2024-04-23' '2024-04-24' '2024-05-01'
 '2024-05-02' '2024-05-03' '2024-05-06' '2024-05-07' '2024-05-08'
 '2024-05-13' '2024-05-14' '2024-05-15' '2024-05-16' '2024-05-20'
 '2024-05-31' '2024-06-03' '2024-06-04' '2024-06-05' '2024-06-06'
 '2024-06-10' '2024-06-11' '2024-06-12' '2024-06-13' '2024-06-20'
 '2024-06-25' '2024-06-27' '2024-07-03' '2024-07-11' '2024-07-12'
 '2024-07-16' '2024-07-17' '2024-07-22' '2024-07-23' '2024-07-24'
 '2024-07-25' '2024-07-31' '2024-08-01' '2024-08-12' '2024-08-13'
 '2024-08-14' '2024-08-15' '2024-08-20' '2024-08-26' '2024-08-28'
 '2024-08-30' '2024-08-31' '2024-09-02' '2024-09-10']
['3' '2']


In [57]:
# convert CreatedDateUtc to datetime format
df['CreatedDateUtc_'] = pd.to_datetime(df['CreatedDateUtc'])
# Extract the day of week
df['DayOfWeek'] = df['CreatedDateUtc_'].dt.day_name()

print(df['DayOfWeek'].unique())
print(df['DayOfWeek'].value_counts())



['Tuesday' 'Thursday' 'Wednesday' 'Friday' 'Monday' 'Saturday']
DayOfWeek
Tuesday      102
Wednesday     34
Thursday      32
Monday        30
Friday        10
Saturday       2
Name: count, dtype: int64


In [58]:
# One hot encoding the days
df_encoded = pd.get_dummies(df, columns=['DayOfWeek'])

df_encoded.drop('CreatedDateUtc', axis='columns', inplace=True)
df_encoded.head()

,short_description,priority,CreatedDateUtc_Short,CreatedDateUtc_,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
0,avscmawapp04 alerting in Solarwinds,3,2024-04-16,2024-04-16 07:39:05.380584100+00:00,False,False,False,False,True,False
1,SAP - Azure Monitoring Agent reinstall for Pro...,3,2024-04-16,2024-04-16 07:39:05.416672+00:00,False,False,False,False,True,False
2,NaN,3,2024-04-16,2024-04-16 07:39:07.774185+00:00,False,False,False,False,True,False
3,NaN,3,2024-04-16,2024-04-16 07:39:07.834434200+00:00,False,False,False,False,True,False
4,eu2qasltes21 node down in Solarwinds,3,2024-04-16,2024-04-16 07:39:09.791338400+00:00,False,False,False,False,True,False


In [59]:
#Only needed features extracted
df.to_csv("Extracted Features.csv", index=False)